In [1]:
# Import dependencies 
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sqlalchemy import create_engine

In [2]:
# Load data from Resources as tab seperated file into dataframe
pd.set_option('display.max_columns', None)
mental_df = pd.read_csv('./Resources/cleaned_data.csv')
mental_df

,Does your employer provide mental health benefits as part of healthcare coverage?,Do you know the options for mental health care available under your employer-provided coverage?,Does your employer offer resources to learn more about mental health concerns and options for seeking help?,If a mental health issue prompted you to request a medical leave from work asking for that leave would be:,Do you think that discussing a mental health disorder with your employer would have negative consequences?,Do you think that discussing a physical health issue with your employer would have negative consequences?,Do you feel that your employer takes mental health as seriously as physical health?,Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?,Do you have previous employers?,Have your previous employers provided mental health benefits?,Were you aware of the options for mental health care provided by your previous employers?,Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?,Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?,Do you think that discussing a mental health disorder with previous employers would have negative consequences?,Would you have been willing to discuss a mental health issue with your direct supervisor(s)?,Did you feel that your previous employers took mental health as seriously as physical health?,Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?,Would you bring up a mental health issue with a potential employer in an interview?,Do you feel that being identified as a person with a mental health issue would hurt your career?,Do you think that team members/co-workers would view you more negatively if they knew you suffered from a mental health issue?,How willing would you be to share with friends and family that you have a mental illness?,Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?,Do you have a family history of mental illness?,Have you had a mental health disorder in the past?,Do you currently have a mental health disorder?,Have you been diagnosed with a mental health condition by a medical professional?,Have you ever sought treatment for a mental health issue from a mental health professional?,If you have a mental health issue do you feel that it interferes with your work when being treated effectively?,If you have a mental health issue do you feel that it interferes with your work when NOT being treated effectively?,What is your gender?,What country do you live in?,What country do you work in?
0,No,Yes,Yes,Somewhat easy,No,No,Yes,No,1,Yes they all did,I was aware of some,None did,Yes always,None of them,Some of my previous employers,Some did,None of them,No,No I don't think it would,No I don't think they would,Somewhat open,No,Yes,Yes,Yes,Yes,1,Rarely,Sometimes,Male,United States of America,United States of America
1,Yes,Yes,No,Neither easy nor difficult,Yes,Maybe,No,No,1,I don't know,N/A (not currently aware),Some did,I don't know,Some of them,Some of my previous employers,Some did,Some of them,No,Yes I think it would,Maybe,Somewhat open,Yes I experienced,Yes,Yes,Yes,Yes,1,Sometimes,Sometimes,Female,United States of America,United States of America
2,Yes,I am not sure,Yes,Somewhat easy,Yes,Yes,No,Yes,1,No none did,Yes I was aware of all of them,None did,I don't know,Yes all of them,No at none of my previous employers,None did,Some of them,Maybe,Yes I think it would,Maybe,Somewhat open,Yes I experienced,No,No,Yes,No,1,Not applicable to me,Often,Male,United Kingdom,United Kingdom
3,I don't know,No,No,Somewhat easy,No,No,Yes,No,1,Some did,I was aware of some,None did,I don't know,None of them,Yes at all of my pre

# Creating tables with categorical variables for database

In [3]:
# build a dataframe of the questions
dict = {}
count = 0
for column in mental_df.columns:
    
    dict[count] = [column]
    count = count + 1
    
    
dict

questions_df = pd.DataFrame.from_dict(dict, orient='index', columns=['Question'])
questions_df

,Question
0,Does your employer provide mental health benef...
1,Do you know the options for mental health care...
2,Does your employer offer resources to learn mo...
3,If a mental health issue prompted you to reque...
4,Do you think that discussing a mental health d...
5,Do you think that discussing a physical health...
6,Do you feel that your employer takes mental he...
7,Have you heard of or observed negative consequ...
8,Do you have previous employers?
9,Have your previous employers provided mental h...


In [4]:
# rename the columns in the dataframe to numbers
column_rename = {}
count = 0
for column in mental_df.columns:
    
    column_rename[column] = count
    count = count + 1
    
    
column_rename

renamed_df = mental_df.rename(columns=column_rename)
renamed_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,No,Yes,Yes,Somewhat easy,No,No,Yes,No,1,Yes they all did,I was aware of some,None did,Yes always,None of them,Some of my previous employers,Some did,None of them,No,No I don't think it would,No I don't think they would,Somewhat open,No,Yes,Yes,Yes,Yes,1,Rarely,Sometimes,Male,United States of America,United States of America
1,Yes,Yes,No,Neither easy nor difficult,Yes,Maybe,No,No,1,I don't know,N/A (not currently aware),Some did,I don't know,Some of them,Some of my previous employers,Some did,Some of them,No,Yes I think it would,Maybe,Somewhat open,Yes I experienced,Yes,Yes,Yes,Yes,1,Sometimes,Sometimes,Female,United States of America,United States of America
2,Yes,I am not sure,Yes,Somewhat easy,Yes,Yes,No,Yes,1,No none did,Yes I was aware of all of them,None did,I don't know,Yes all of them,No at none of my previous employers,None did,Some of them,Maybe,Yes I think it would,Maybe,Somewhat open,Yes I experienced,No,No,Yes,No,1,Not applicable to me,Often,Male,United Kingdom,United Kingdom
3,I don't know,No,No,Somewhat easy,No,No,Yes,No,1,Some did,I was aware of some,None did,I don't know,None of them,Yes at all of my previous employers,Some did,None of them,Yes,Yes I think it would,No I don't think they would,Not applicable to me (I do not have a mental i...,No,No,No,No,No,0,Not applicable to me,Not applicable to me,Male,United States of America,United States of America
4,Yes,Yes,Yes,Very easy,No,No,I don't know,No,1,Some did,I was aware of some,Some did,Sometimes,Some of them,Some of my previous employers,Some did,Some of them,No,Maybe,Maybe,Somewhat open,Yes I observed,Yes,Yes,Yes,Yes,1,Sometimes,Often,Female,United States of America,United States of America
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,Yes,I am not sure,Yes,Somewhat easy,No,No,Yes,No,1,Some did,No I only became aware later,Some did,Yes always,Some of them,No at none of my previous employers,Some did,None of them,No,Yes I think it would,Maybe,Somewhat not open,No,I don't know,Maybe,Maybe,No,0,Rarely,Rarely,Male,Canada,Canada
857,I don't know,I am not sure,I don't know,Somewhat easy,Maybe,No,I don't know,No,1,I don't know,N/A (not currently aware),I don't know,I don't know,Some of them,Some of my previous employers,Some did,None of them,Maybe,Yes it has,Maybe,Somewhat open,Yes I experienced,I don't know,Yes,Yes,Yes,1,Rarely,Often,Female,Canada,Canada
858,Yes,No,No,Somewhat easy,No,No,Yes,No,1,Some did,I was aware of some,None did,I don't know,Some of them,No at none of my previous employers,None did,Yes all of them,No,Maybe,No I don't think they would,Very open,Yes I experienced,Yes,Yes,Yes,Yes,1,Rarely,Often,Female,United States of America,United States of America
859,Yes,Yes,Yes,Somewhat difficult,Maybe,Maybe,I don't know,Yes,1,Some did,I was aware of some,None did,Sometimes,Yes all of them,Some of my previous employers,None did,Some of them,No,Yes it has,No I don't think they would,Somewhat open,Yes I observed,Yes,Yes,Maybe,Yes,1,Rarely,Sometimes,Male,United States of America,United States of America


# Create Database

In [19]:
# Create an in-memory SQLite database

# Create the engine
engine = create_engine("sqlite:///mental_health.db", echo=False)

# Reflect existing db into a new model
Base = automap_base()
# Refelct the tables
Base.prepare(engine, reflect=True)

In [20]:
# Use pandas to_sql to write records stored in a dataframe to a SQL database

# Write dataframe of all survey questions to a sqlite table
renamed_df.to_sql('pre_encoded', con=engine, if_exists='replace')
# Write dataframe of all questions with encoded numbers to a sqlite table
questions_df.to_sql('pre_encoded_questions', con=engine, if_exists='replace')

In [21]:
# Check that you can select a table from the database
#engine.execute("SELECT * FROM pre_encoded").fetchall()
engine.execute("SELECT * FROM pre_encoded_questions").fetchall()

[(0, 'Does your employer provide mental health benefits as part of healthcare coverage?'),
 (1, 'Do you know the options for mental health care available under your employer-provided coverage?'),
 (2, 'Does your employer offer resources to learn more about mental health concerns and options for seeking help?'),
 (3, 'If a mental health issue prompted you to request a medical leave from work asking for that leave would be:'),
 (4, 'Do you think that discussing a mental health disorder with your employer would have negative consequences?'),
 (5, 'Do you think that discussing a physical health issue with your employer would have negative consequences?'),
 (6, 'Do you feel that your employer takes mental health as seriously as physical health?'),
 (7, 'Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?'),
 (8, 'Do you have previous employers?'),
 (9, 'Have your previous employers provided mental health benefit